In [1]:
mode = "Live" # Backtesting/Live/Algo

%run "KiteConnect_Library.ipynb"   
%run "KiteConnect_Strategies.ipynb" 

warnings.filterwarnings('ignore')



In [2]:
#%run "KiteConnect_Strategies.ipynb" 
STATE = ""
openpositions = 0
tradelogger.setLevel(logging.DEBUG)

%%javascript
IPython.notebook.kernel.execute("URL = '" + window.location + "'");

In [3]:
portfolio = pd.DataFrame(downloadlist.tolist())
portfolio = pd.DataFrame(['CIPLA','INFRATEL','BHARTIARTL'])

portfolioToken = portfolio[0].apply(EQTOKEN).tolist()

In [4]:
portfolio[0].iloc[-1]

'BHARTIARTL'

In [5]:
papertrade = True

In [6]:
if mode == "Live":
    kws.connect(threaded=True)

In [7]:
if mode == "Live":
    print(kws.is_connected())

True


In [8]:
if mode == "Live":
    kws.close()

In [91]:
def buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    tradelogger.info("BUY BO: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol, exchange=kite.EXCHANGE_NSE, transaction_type=kite.TRANSACTION_TYPE_BUY,
                        order_type=kite.ORDER_TYPE_SL, product=kite.PRODUCT_MIS, variety=kite.VARIETY_BO, 
                                quantity=quantity, trigger_price=trigger, price=price,
                                squareoff=squareoff,  stoploss=stoploss )
        tradelogger.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        tradelogger.info("Order placement failed: {}".format(e.message))



def sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1): 
    tradelogger.info("SELL BO: "+symbol+", price: "+str(price)+", squareoff: "+str(squareoff)+", stoploss: "+str(stoploss)+", quantity: "+str(quantity))
    if papertrade:
        return
    
    try:
        order_id = kite.place_order(tradingsymbol=symbol, exchange=kite.EXCHANGE_NSE, transaction_type=kite.TRANSACTION_TYPE_SELL,
                                order_type=kite.ORDER_TYPE_SL, product=kite.PRODUCT_MIS, variety=kite.VARIETY_BO,
                                quantity=quantity, trigger_price=trigger, price=price,
                                stoploss=stoploss, squareoff=squareoff )
        tradelogger.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        tradelogger.info("Order placement failed: {}".format(e.message))

In [110]:
symbol = 'BHARTIARTL'
symbolToken = EQTOKEN('BHARTIARTL')

price = kite.ltp(symbolToken)[str(symbolToken)]['last_price']
toTick = lambda x,n=5: np.round((np.floor(x *100)+n-1)/n)*n/100

sl = toTick(price * 0.001)
target = toTick(price * 0.01)

In [111]:
papertrade = False
buy_bo(symbol, price=price, trigger=price, stoploss=sl, squareoff=target, quantity=1)

In [ ]:
papertrade = False
sell_bo(symbol, price, price, sl, target, 1)

In [92]:
price-sl

315.05

In [107]:
orders_df = pd.DataFrame(kite.orders())
open_orders = orders_df[orders_df['status']=='TRIGGER PENDING']

In [108]:
parent_order_id = open_orders['parent_order_id'].iloc[0]
order_id = open_orders['order_id'].iloc[0]

In [109]:
kite.exit_order(variety=kite.VARIETY_BO, order_id=order_id, parent_order_id=parent_order_id)



In [94]:
open_orders

,average_price,cancelled_quantity,disclosed_quantity,exchange,exchange_order_id,exchange_timestamp,exchange_update_timestamp,filled_quantity,guid,instrument_token,...,product,quantity,status,status_message,tag,tradingsymbol,transaction_type,trigger_price,validity,variety
16,0.0,0,1,NSE,1000000003411529,2019-04-24 12:12:07,None,0,None,2714625,...,BO,1,OPEN,None,None,BHARTIARTL,SELL,0.0,DAY,bo
